# Data mining assignment 2

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random as r
import lightgbm as lgb

from collections import Counter
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
df = pd.read_csv('training_set_vu_DM.csv')
df_test = pd.read_csv('test_set_vu_DM.csv')

In [3]:
train_vals = max(df.count().values)
test_vals = max(df_test.count().values)

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

Veel missing values, verwijder alle features met meer dan 10% NA's (kan evt nog meer NA's worden, maar dat wordt wel meer imputen)

In [5]:
null_values = df.isnull().sum()
to_keep = null_values.index[np.asarray(null_values/train_vals < 0.90)]
df_no_na = df[to_keep]

In [6]:
to_keep_test = [x for x in list(to_keep) if x not in ['position', 'click_bool', 'booking_bool']]
df_test2 = df_test[to_keep_test]

In [7]:
comp_rate = pd.Series(np.zeros(train_vals,))
comp_test_rate = pd.Series(np.zeros(test_vals,))
for x in range(1,9):
    temp_rate = np.where(df['comp'+str(x)+'_rate'].isnull(), 0, df['comp'+str(x)+'_rate'])
    comp_rate += temp_rate
    
    temp_test_rate = np.where(df_test['comp'+str(x)+'_rate'].isnull(), 0, df_test['comp'+str(x)+'_rate'])
    comp_test_rate += temp_test_rate
comp_rate = np.where(comp_rate < -1, -1, comp_rate)
comp_rate = pd.Series(np.where(comp_rate > 1, 1, comp_rate))

comp_test_rate = np.where(comp_test_rate < -1, -1, comp_test_rate)
comp_test_rate = pd.Series(np.where(comp_test_rate > 1, 1, comp_test_rate))

comp_inv = pd.Series(np.zeros(train_vals,))
comp_test_inv = pd.Series(np.zeros(test_vals,))
for x in range(1,9):
    temp_inv = np.where(df['comp'+str(x)+'_inv'].isnull(), 0, df['comp'+str(x)+'_inv'])
    comp_inv += temp_inv
    
    temp_test_inv = np.where(df_test['comp'+str(x)+'_inv'].isnull(), 0, df_test['comp'+str(x)+'_inv'])
    comp_test_inv += temp_test_inv
    

df_no_na['comp_rate'] = comp_rate
df_no_na['comp_inv'] = comp_inv

df_test2['comp_rate'] = comp_test_rate
df_test2['comp_inv'] = comp_test_inv
df_no_na2 = df_no_na.drop(['comp2_rate', 'comp3_rate', 'comp5_rate', 'comp8_rate',
                           'comp2_inv', 'comp3_inv', 'comp5_inv', 'comp8_inv',
                          'comp5_rate_percent_diff', 'comp8_rate_percent_diff', 'comp2_rate_percent_diff'], axis=1)
df_test3 = df_test2.drop(['comp2_rate', 'comp3_rate', 'comp5_rate', 'comp8_rate',
                           'comp2_inv', 'comp3_inv', 'comp5_inv', 'comp8_inv',
                          'comp5_rate_percent_diff', 'comp8_rate_percent_diff', 'comp2_rate_percent_diff'], axis=1)
df_test3.isnull().sum()

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

srch_id                              0
date_time                            0
site_id                              0
visitor_location_country_id          0
prop_country_id                      0
prop_id                              0
prop_starrating                      0
prop_review_score                 7266
prop_brand_bool                      0
prop_location_score1                 0
prop_location_score2           1088032
prop_log_historical_price            0
price_usd                            0
promotion_flag                       0
srch_destination_id                  0
srch_length_of_stay                  0
srch_booking_window                  0
srch_adults_count                    0
srch_children_count                  0
srch_room_count                      0
srch_saturday_night_bool             0
orig_destination_distance      1608679
random_bool                          0
comp_rate                            0
comp_inv                             0
dtype: int64

Prop_review_score en Prop_location_score2 hebben nog NA's dus die moeten we imputen. Heb ze nu allebei ge-impute aan de hand van hun distributie, maar we kunnen misschien ook nog prop_location_score2 imputen adhv de andere location score maar weet niet echt goed hoe

In [8]:
probs = df_no_na2['prop_review_score'].value_counts(normalize=True)
isnull = df_no_na2['prop_review_score'].isnull()
np.random.seed(15)
df_no_na2.loc[isnull, 'prop_review_score'] = np.random.choice(probs.index, size=len(df[isnull]), p=probs.values)

probs = df_test3['prop_review_score'].value_counts(normalize=True)
isnull = df_test3['prop_review_score'].isnull()
np.random.seed(15)
df_test3.loc[isnull, 'prop_review_score'] = np.random.choice(probs.index, size=len(df_test[isnull]), p=probs.values)

In [9]:
df_no_na2['prop_location_score2'] = df['prop_location_score2']

probs = df_no_na2['prop_location_score2'].value_counts(normalize=True)
isnull = df_no_na2['prop_location_score2'].isnull()
np.random.seed(15)
df_no_na2.loc[isnull, 'prop_location_score2'] = np.random.choice(probs.index, size=len(df[isnull]), p=probs.values)

df_test3['prop_location_score2'] = df_test['prop_location_score2']

probs = df_test3['prop_location_score2'].value_counts(normalize=True)
isnull = df_test3['prop_location_score2'].isnull()
np.random.seed(15)
df_test3.loc[isnull, 'prop_location_score2'] = np.random.choice(probs.index, size=len(df_test[isnull]), p=probs.values)

In [10]:
df_no_na2 = df_no_na2.drop(['orig_destination_distance'], axis=1)
df_test3 = df_test3.drop(['orig_destination_distance'], axis=1)

In [11]:
df_no_na2 = df_no_na2.drop(['date_time'], axis=1)
df_test3 = df_test3.drop(['date_time'], axis=1)

In [12]:
df_no_na2['relevance'] = np.asarray(4*df_no_na2['booking_bool'] + df_no_na2['click_bool'])

In [13]:
r.seed(15)
# Train and validation data splitten op basis van search id
train_ids = np.asarray(r.sample(list(df_no_na2['srch_id'].unique()), round(0.2*df_no_na2['srch_id'].unique().size)))
val_ids = np.setdiff1d(df_no_na2['srch_id'].unique(), train_ids)

In [14]:
train_data = df_no_na2.loc[df_no_na2['srch_id'].isin(train_ids)]
val_data = df_no_na2.loc[df_no_na2['srch_id'].isin(val_ids)]

In [15]:
# train_data.head()

In [16]:
y_train = train_data['relevance']
X_train = train_data.drop(['relevance', 'click_bool', 'booking_bool', 'position'], axis=1)
y_val = val_data['relevance']
X_val = val_data.drop(['relevance', 'click_bool', 'booking_bool', 'position'], axis=1)


In [17]:
train_queries = list(Counter(np.asarray(X_train['srch_id'])).values())
val_queries = list(Counter(np.asarray(X_val['srch_id'])).values())

In [18]:
gbm = lgb.LGBMRanker()

In [19]:
gbm.fit(X_train, y_train, group=train_queries, 
        eval_set=[(X_val, y_val)], eval_group = [val_queries], 
        eval_at=[5, 10, 25], early_stopping_rounds=50)

[1]	valid_0's ndcg@5: 0.25029	valid_0's ndcg@10: 0.321714	valid_0's ndcg@25: 0.398659
Training until validation scores don't improve for 50 rounds
[2]	valid_0's ndcg@5: 0.277225	valid_0's ndcg@10: 0.349523	valid_0's ndcg@25: 0.422296
[3]	valid_0's ndcg@5: 0.292673	valid_0's ndcg@10: 0.363824	valid_0's ndcg@25: 0.433803
[4]	valid_0's ndcg@5: 0.30209	valid_0's ndcg@10: 0.373175	valid_0's ndcg@25: 0.441523
[5]	valid_0's ndcg@5: 0.310379	valid_0's ndcg@10: 0.380915	valid_0's ndcg@25: 0.447858
[6]	valid_0's ndcg@5: 0.315876	valid_0's ndcg@10: 0.385726	valid_0's ndcg@25: 0.451909
[7]	valid_0's ndcg@5: 0.317493	valid_0's ndcg@10: 0.387636	valid_0's ndcg@25: 0.453193
[8]	valid_0's ndcg@5: 0.319495	valid_0's ndcg@10: 0.389502	valid_0's ndcg@25: 0.454665
[9]	valid_0's ndcg@5: 0.321429	valid_0's ndcg@10: 0.391283	valid_0's ndcg@25: 0.456054
[10]	valid_0's ndcg@5: 0.322676	valid_0's ndcg@10: 0.392495	valid_0's ndcg@25: 0.45697
[11]	valid_0's ndcg@5: 0.324484	valid_0's ndcg@10: 0.394219	valid_0's n

[94]	valid_0's ndcg@5: 0.339088	valid_0's ndcg@10: 0.407944	valid_0's ndcg@25: 0.469397
[95]	valid_0's ndcg@5: 0.339231	valid_0's ndcg@10: 0.40799	valid_0's ndcg@25: 0.469459
[96]	valid_0's ndcg@5: 0.33921	valid_0's ndcg@10: 0.407947	valid_0's ndcg@25: 0.469471
[97]	valid_0's ndcg@5: 0.339068	valid_0's ndcg@10: 0.407799	valid_0's ndcg@25: 0.469353
[98]	valid_0's ndcg@5: 0.338943	valid_0's ndcg@10: 0.407713	valid_0's ndcg@25: 0.469261
[99]	valid_0's ndcg@5: 0.338945	valid_0's ndcg@10: 0.407854	valid_0's ndcg@25: 0.469368
[100]	valid_0's ndcg@5: 0.339057	valid_0's ndcg@10: 0.40799	valid_0's ndcg@25: 0.469449
Did not meet early stopping. Best iteration is:
[95]	valid_0's ndcg@5: 0.339231	valid_0's ndcg@10: 0.40799	valid_0's ndcg@25: 0.469459


LGBMRanker(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
      importance_type='split', learning_rate=0.1, max_depth=-1,
      min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
      n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
      random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
      subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [21]:
test_pred = gbm.predict(df_test3)

In [30]:
df_test3['relevance'] = test_pred

In [36]:
df_test3.sort_values(by=['srch_id', 'relevance'], ascending=[True, False], inplace=True)

In [37]:
df_test3.head()

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,comp_rate,comp_inv,relevance
23,1,24,216,219,99484,3,4.000,1,2.400,0.218,4.540,69.000,1,19222,1,10,2,0,1,0,0,1.000,0.000,1.035
12,1,24,216,219,61934,3,4.500,1,2.890,0.242,4.690,88.880,1,19222,1,10,2,0,1,0,0,0.000,0.000,0.875
9,1,24,216,219,54937,3,4.000,1,2.080,0.165,4.750,83.300,1,19222,1,10,2,0,1,0,0,0.000,0.000,0.842
4,1,24,216,219,24194,3,4.500,1,2.940,0.209,4.720,79.000,0,19222,1,10,2,0,1,0,0,0.000,0.000,0.520
6,1,24,216,219,34263,3,4.500,1,3.090,0.130,4.630,79.000,0,19222,1,10,2,0,1,0,0,0.000,0.000,0.362


In [38]:
kaggle_answer = pd.DataFrame({'srch_id': df_test3['srch_id'],
                             'prop_id': df_test3['prop_id']})
kaggle_answer.to_csv('expedia_answer.csv', index=False)

## Tot hierboven is wat ik nu gedaan heb, rest is nog van daarvoor

Geen NA's meer nu. Is nog wel te veel data om chill mee te kunnen werken, dus heb de data naar een vijfde gedownsampled, dus van 199795 naar 40000 search id's. Is denk ik nog steeds genoeg data om goed mee te kunnen werken en we kunnen op het einde wel weer op de volledige data het model trainen

In [ ]:
a = np.asarray(list(set(df_no_na['srch_id'].values))) # 199795 unique searches

np.random.seed(15)
subset_users = np.random.choice(a, size=40000, replace=False) # downsample to 40000

subset_df = df_no_na.loc[df_no_na['srch_id'].isin(subset_users)]
subset_df.info() # van 4958347 naar 996086 values

In [ ]:
subset_df.describe()

In [ ]:
corr_matrix = subset_df.corr()

In [ ]:
corr_matrix['click_bool']

In [ ]:
print(subset_df['click_bool'].value_counts(normalize=True))
print(subset_df['booking_bool'].value_counts(normalize=True))

Op maar 4.4% van de hotels wordt geklikt, en 2.8% krijgt een booking

In [ ]:
sns.barplot(subset_df['prop_starrating'], subset_df['click_bool'])

Er wordt het meest geklikt op een 4-sterren hotel, maar alsnog maar 5% van die hotels.

In [ ]:
clicked = subset_df[subset_df['click_bool'] == 1]

sns.barplot(clicked['position'], clicked['booking_bool'])

Hogere positie = meer kans om geboekt te worden bij hotels waarop geklikt is